Any columns that all statistics are computed for will have fresher data and more detail.

This is not meant to be a production-capable notebook but an example so the approach is clear.

This can take a while for big tables.

Used successfully on internal POCs to quickly get all metadata added to tables for comment generation.

In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from typing import List, Optional
import os
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

def compute_statistics_for_tables_from_csv(
    csv_path: str, 
    table_column_name: str = "table_names",
    skip_errors: bool = True,
    limit: Optional[int] = None
) -> List[str]:
    """
    Read table names from a CSV file and compute statistics for all columns in each table.
    
    Args:
        csv_path (str): Path to the CSV file containing table names
        table_column_name (str): Name of the column in the CSV that contains table names
        skip_errors (bool): Whether to continue processing if an error occurs for a table
        limit (Optional[int]): Maximum number of tables to process, None for all
        
    Returns:
        List[str]: List of tables that were successfully processed
    """

    spark = SparkSession.builder.getOrCreate() ### Initialize Spark session in case this is not being run in a notebook.
    
    # if not os.path.exists(csv_path):
    #     error_msg = f"CSV file not found at path: {csv_path}"
    #     logger.error(error_msg)
    #     raise FileNotFoundError(error_msg)
    
    try:
        tables_df = spark.read.csv(csv_path, header=True, inferSchema=True)

        if table_column_name not in tables_df.columns:
            available_columns = ", ".join(tables_df.columns)
            error_msg = f"Column '{table_column_name}' not found in CSV. Available columns: {available_columns}"
            logger.error(error_msg)
            raise ValueError(error_msg)
            
        tables_pd = tables_df.toPandas()
        
    except Exception as e:
        logger.warning(f"Failed to read CSV with Spark, falling back to pandas: {str(e)}")
        try:
            tables_pd = pd.read_csv(csv_path)
            if table_column_name not in tables_pd.columns:
                available_columns = ", ".join(tables_pd.columns)
                error_msg = f"Column '{table_column_name}' not found in CSV. Available columns: {available_columns}"
                logger.error(error_msg)
                raise ValueError(error_msg)
        except Exception as e2:
            error_msg = f"Failed to read CSV file: {str(e2)}"
            logger.error(error_msg)
            raise RuntimeError(error_msg)
    
    table_names = tables_pd[table_column_name].dropna().unique().tolist()
    logger.info(f"Found {len(table_names)} unique table names in CSV")
    
    if limit is not None and limit > 0:
        table_names = table_names[:limit]
        logger.info(f"Limited processing to {limit} tables")

    successful_tables = []
    failed_tables = []
    
    for i, full_table_name in enumerate(table_names):
        try:
            logger.info(f"Processing table {i+1}/{len(table_names)}: {full_table_name}")
            try:
                spark.catalog.tableExists(full_table_name)
            except Exception as e:
                logger.warning(f"Table validation failed for {full_table_name}: {str(e)}")
                if not skip_errors:
                    raise
                failed_tables.append(full_table_name)
                continue
            
            logger.info(f"Computing statistics for {full_table_name}")
            spark.sql(f"ANALYZE TABLE {full_table_name} COMPUTE STATISTICS FOR ALL COLUMNS")
            
            successful_tables.append(full_table_name)
            logger.info(f"Successfully computed statistics for {full_table_name}")
            
        except Exception as e:
            error_msg = f"Error processing table {full_table_name}: {str(e)}"
            logger.error(error_msg)
            failed_tables.append(full_table_name)
            if not skip_errors:
                raise RuntimeError(error_msg)
    
    logger.info(f"Processing complete. Successfully processed {len(successful_tables)} tables.")
    if failed_tables:
        logger.warning(f"Failed to process {len(failed_tables)} tables: {', '.join(failed_tables)}")
    
    return successful_tables

def main():
    try:
        try:
            dbutils.widgets.text("csv_path", "table_names.csv", "Path to CSV file with table names")
            dbutils.widgets.text("table_column_name", "table_names", "Column name containing table names")
            dbutils.widgets.dropdown("skip_errors", "True", ["True", "False"], "Skip errors and continue processing")
            dbutils.widgets.text("limit", "10", "Maximum number of tables to process (optional)")
            
            csv_path = os.path.join(os.getcwd(), dbutils.widgets.get("csv_path"))
            table_column_name = dbutils.widgets.get("table_column_name")
            skip_errors = dbutils.widgets.get("skip_errors") == "True"
            limit_str = dbutils.widgets.get("limit")
            limit = int(limit_str) if limit_str.strip() else None
        except NameError:
            logger.info("Not running in a notebook environment, using default values")
            csv_path = os.path.join(os.path.dirname(os.getcwd()), dbutils.widgets.get("csv_path"))
            table_column_name = "table_name"
            skip_errors = True
            limit = None
        
        successful_tables = compute_statistics_for_tables_from_csv(
            csv_path=csv_path,
            table_column_name=table_column_name,
            skip_errors=skip_errors,
            limit=limit
        )
        
        try:
            display(spark.createDataFrame([(table,) for table in successful_tables], ["Processed Tables"]))
        except NameError:
            print(f"Processed tables: {successful_tables}")
            
    except Exception as e:
        logger.error(f"Error in main function: {str(e)}")
        raise

if __name__ == "__main__":
    main()
